# Lab 5 - potoki transformujące zbiory danych

Na podstawie materiałów zawartych w labach 4-6 można z łatwością zauważyć, że kolejność wykonania operacji jest niezwykle istotna. W przypadku dużych zbiorów danych, wymagających szeroko zakrojonych operacji transformacji, zapanowanie nad kodem i kolejnością wykonywania operacji może być problematyczne.

Rozwiązaniem problemu są potoki transformujące z biblioteki **Scikit-learn**.

Dzięki potokom można z łatwością utrzymywać kod w sposób modularny, co oznacza, że można z łatwością dzielić zadania na mniejsze etapy. Potoki transformujące pomagają unikać zjawiska wycieków informacji z danych treningowych do danych testowych za sprawą izolowania poznanych transformacji do danych treningowych, a następnie stosowanie tych samych transformacji do danych testowych lub walidacyjnych. Zastosowanie takich operacji optymalizujących transformacje wpływają pozytywnie na oszczędność czasu potrzebnego na ogarnięcie dużych fragmentów kodu, a także na walkę z późniejszymi błędami.

## Stosowanie gotowych transformatorów

Stosowanie potoków transformujących polega na utworzeniu instancji klasy **Pipeline**, której inicjalizator przyjmuje listę zawierającą sprecyzowane kroki przetwarzające dane w postaci krotek: (nazwa, estymator). W znacznej części przypadków wystarczające pozostają klasy (np. *SimpleImputer*) dostarczane przez bibliotekę **Scikit-learn**. Warto mieć na uwadze fakt, że estymator musi być klasą która zawiera metody *fit* oraz *transform*.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
num_values_pipeline = Pipeline([
    ('impute_missing_values', SimpleImputer(strategy='mean')),
    ('scale_values', MinMaxScaler()),
])

Alternatywą jest zastosowanie funkcji *make_pipeline*, która przyjmuje dowolną liczbę parametrów w postaci estymatorów. Warto zauważyć, że w tym przypadku nie występuje konieczność przekazania nazw poszczególnych kroków.

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
num_values_pipeline = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(),
)

Zastosowanie potoku na zbiorze danych wymaga wywołania kolejno metod: *fit* i *transform* lub metody *fit_transform*.

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
data = fetch_california_housing(as_frame=True)['frame']

In [ ]:
data

In [ ]:
num_values_pipeline.fit_transform(data)

In [ ]:
import pandas as pd

In [ ]:
data_preprocessed = pd.DataFrame(
    num_values_pipeline.fit_transform(data),
    columns=num_values_pipeline.get_feature_names_out(),
    index=data.index,
)

In [ ]:
data_preprocessed

## Potoki dopasowane do typów danych w atrybutach

W przypadku zbiorów danych zawierających różne typy wartości w atrybutach (np. numeryczne i symboliczne), stosowanie potoków uzupełniających wartości wybrakowane za pomocą średniej arytmetycznej może być problematyczne. Rozwiązaniem problemu w takiej sytuacji jest klasa **ColumnTransformer**, która oprócz listy zawierającej nazwe i estymator, przyjmuje także listę nazw atrybutów, na których dany krok ma zostać zastosowany.

In [ ]:
from sklearn.datasets import fetch_kddcup99

In [ ]:
data = fetch_kddcup99(as_frame=True)['frame']

In [ ]:
data

In [ ]:
data[['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent']] = data[['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent']].astype(int)

In [ ]:
data[['protocol_type', 'service', 'flag', 'labels']] = data[['protocol_type', 'service', 'flag', 'labels']].applymap(lambda x: x.decode('utf-8'))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
cat_values_pipeline = make_pipeline(
    OrdinalEncoder(handle_unknown='error'),
)

In [ ]:
preprocessing_pipeline = ColumnTransformer([
    ('num_attributes_steps', num_values_pipeline, data.select_dtypes('number').columns),
    ('cat_attributes_steps', cat_values_pipeline, ('protocol_type', 'service', 'flag', 'labels')),
])

Alternatywnie, jak w przypadku funkcji *make_pipeline*, zastosowanie funkcji *make_column_transformer* pozwoli na pominięcie wskazania nazwy kroku. Wartym uwagi dodatkiem jest funkcja *make_column_selector*, która wybierze atrybuty o wskazanym typie.

In [ ]:
from sklearn.compose import make_column_selector, make_column_transformer

In [ ]:
preprocessing_pipeline = make_column_transformer(
    (num_values_pipeline, make_column_selector(dtype_include='number')),
    (cat_values_pipeline, ('protocol_type', 'service', 'flag', 'labels')),
)

In [ ]:
data_preprocessed = pd.DataFrame(
    preprocessing_pipeline.fit_transform(data),
    columns=preprocessing_pipeline.get_feature_names_out(),
    index=data.index,
)

In [ ]:
data_preprocessed

## Zadania

1. Dokonać refaktoryzacji zadań z labów 4-6 w taki sposób, aby ich implementacja została zrealizowana całkowicie za pomocą potoków.